In [ ]:
"""
Created on November 18th 2022 at 5:07 PM

By
Sanjusha,Uday kumar,Manvitha,Abhinay 

"""
import pandas as pd
import numpy as np
import csv
from gurobipy import *

# ***READING THE DATA FROM INPUT FILE***

Biginstance_File = open("biginstance.gr","r") # This opens the data from given .gr input file

Noised_Data = Biginstance_File.readlines()[6:] # This reads all the lines starting from Line 7

Nodes_Line = Noised_Data[0].strip().split(" ")
Nodes = int(Nodes_Line[2]) # This gives the value for Number of Nodes

# The following code reads the data from the input file and stores the arcs into a tuple list and respective costs into a dictionary
Arcs_Cost_Data = Noised_Data[1:]
Arcs_Cost = []
for i in Arcs_Cost_Data:
    i=i.strip().split(" ")
    Arcs_Cost.append(i)
Origin = int(input("Enter the Source Node:"))
Arcs = tuplelist()
Costs = {}
for i in Arcs_Cost:
    From_Node = int(i[1])
    To_Node = int(i[2])
    Cost = float(i[3])
    Arcs.append((From_Node,To_Node))
    Costs[From_Node,To_Node] = Cost
    
Biginstance_File.close()

# ***USING GUROBI SOLVER TO PERFORM OPTIMIZATON***

# Creating a New Model
m = Model('Single Source Shortest Path')

# Defining the Decision Variables and Objective Function
x = m.addVars(Arcs, lb=0, obj=Costs, name ="Network Flow")

# Defining the Constraints
for i in range(1, Nodes+1):
    m.addConstr( sum(x[i,j] for i,j in Arcs.select(i, '*')) - sum(x[j,i] for j,i in Arcs.select('*',i)) == 
                     (Nodes-1 if i==Origin else -1 ),'node%s_' % i )
    
# To formulate the mathematical model in LP format
m.write("Single_Source_Shortest_Path.lp")

#Call Gurobi Optimizer
m.optimize()

# Write Output if termination is optimal
Output=[(Origin," ")] # This results in a output which is a list of tuples
if m.status == GRB.Status.OPTIMAL:
    for i,j in Arcs:
        if(x[i,j].x > 0):
            Output.append((j,i))
            
elif m.status == GRB.INFEASIBLE:
        print('LP is infeasible.')
        
elif m.status == GRB.UNBOUNDED:
        print('LP is unbounded.')

# ***WRITING THE OUTPUT INTO A .CSV FILE***

Output = [list(i) for i in Output] # This converts the list of tuples into list of lists

Data_Frame = pd.DataFrame(list())
Data_Frame.to_csv('Single Source Shortest Path Problem.csv')

with open("Single Source Shortest Path Problem.csv",'w',newline='') as o:
    Out=csv.writer(o)
    Out.writerow(["Node ID","Predecessor ID"])
    Out.writerows(Output)